## Install and import libraries

In [ ]:
%%capture
# !pip install timesfm
!pip install StatsForecast==1.7.4

##Note: If you ran this in Colab, restart the Colab Session afterwards before importing timesfm

## Load Data

In [ ]:
df = pd.read_csv("weekly.csv")

In [ ]:
df = df.drop(["holiday_name", 'shutdown', 'mini_shutdown', 'blackout', 'frankfurt_shutdown', "id"], axis=1)

In [ ]:
df = df.rename(columns={"warehouse": "unique_id", "date": "ds", "orders": "y"})

In [ ]:
df['ds'] = pd.to_datetime(df['ds'])

In [ ]:
df.shape

(688, 12)

## Exploration Data Analysis

In [ ]:
#number of datapoints per time-series
df['unique_id'].value_counts()

,count
unique_id,
Prague_1,172
Prague_2,172
Prague_3,172
Brno_1,172


In [ ]:
data=df.copy()

In [ ]:
df.shape

(688, 12)

In [ ]:
# Since this is a Monash dataset, we use the default settings regarding horizon and seasonality.
horizon = 297
test_size = 297
seasonality=365

# Load preprocessed Data

Y_test_df = pd.read_csv("/content/Future_df_NueralForecast.csv")
df = pd.read_csv("/content/Train_df_NueralForecast.csv")



In [ ]:
df

,Unnamed: 0,unique_id,ds,y,holiday,shops_closed,winter_school_holidays,school_holidays,year,month,day,day_of_week,week_of_year,shops_closed_lag,Shop_opened_after_close
0,1,Brno_1,2020-12-06,5338.0,0,0,0,0,2020,12,6,6,49,0.0,0
1,2,Brno_1,2020-12-07,5621.0,0,0,0,0,2020,12,7,0,50,0.0,0
2,3,Brno_1,2020-12-08,5792.0,0,0,0,0,2020,12,8,1,50,0.0,0
3,4,Brno_1,2020-12-09,5894.0,0,0,0,0,2020,12,9,2,50,0.0,0
4,5,Brno_1,2020-12-10,6122.0,0,0,0,0,2020,12,10,3,50,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4523,1128,Prague_3,2024-01-11,5373.0,0,0,0,0,2024,1,11,3,2,0.0,0
4524,1129,Prague_3,2024-01-12,6373.0,0,0,0,0,2024,1,12,4,2,0.0,0
4525,1130,Prague_3,2024-01-13,6044.0,0,0,0,0,2024,1,13,5,2,0.0,0
4526,1131,Prague_3,2024-01-14,5648.0,0,0,0,0,2024,1,14,6,2,0.0,0


In [ ]:
Y_test_df

,Unnamed: 0,unique_id,ds,holiday,shops_closed,winter_school_holidays,school_holidays,year,month,day,day_of_week,week_of_year,shops_closed_lag,Shop_opened_after_close
0,1133,Brno_1,2024-01-16,0,0,0,0,2024,1,16,1,3,0.0,0
1,1134,Brno_1,2024-01-17,0,0,0,0,2024,1,17,2,3,0.0,0
2,1135,Brno_1,2024-01-18,0,0,0,0,2024,1,18,3,3,0.0,0
3,1136,Brno_1,2024-01-19,0,0,0,0,2024,1,19,4,3,0.0,0
4,1137,Brno_1,2024-01-20,0,0,0,0,2024,1,20,5,3,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1188,Prague_3,2024-03-11,0,0,0,0,2024,3,11,0,11,0.0,0
236,1189,Prague_3,2024-03-12,0,0,0,0,2024,3,12,1,11,0.0,0
237,1190,Prague_3,2024-03-13,0,0,0,0,2024,3,13,2,11,0.0,0
238,1191,Prague_3,2024-03-14,0,0,0,0,2024,3,14,3,11,0.0,0


## TimesFM

Model Specs:
 * TimesFM can accept a **context_length** up to 512 (use 512 if you can, higher=better)
 * **Horizon_length** can be anything - from my own experiments, the model does better with longer horizons
 * The author suggest **horizon_length** <= **context_length**

In [ ]:
import timesfm
tfm = timesfm.TimesFm(
    context_len=512,
    horizon_len=297,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="cpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

(…)oint_1100000/descriptor/descriptor.pbtxt:   0%|          | 0.00/499 [00:00<?, ?B/s]

(…)nts/checkpoint_1100000/metadata/metadata:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

...

checkpoint:   0%|          | 0.00/814M [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 3.38 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 1.38 seconds.
Jitting decoding.
Jitted decoding in 49.15 seconds.


In [ ]:
df["ds"] = pd.to_datetime(df["ds"])

In [ ]:
%%time
forecast_df = tfm.forecast_on_df(
    inputs=df,
    freq='D',
    value_name='y',
    num_jobs=-1,
)

Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.
CPU times: user 20.4 s, sys: 1.99 s, total: 22.4 s
Wall time: 11.8 s


In [ ]:
forecast_df

,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
0,Brno_1,2024-01-16,8740.092773,7959.152832,8235.708984,8433.213867,8612.950195,8740.092773,8892.332031,9067.081055,9263.916016,9571.101562
1,Brno_1,2024-01-17,8796.653320,8018.024414,8270.303711,8482.458008,8635.951172,8796.653320,8965.578125,9148.735352,9358.771484,9686.402344
2,Brno_1,2024-01-18,8852.774414,8038.942383,8334.814453,8532.513672,8709.117188,8852.774414,9016.925781,9202.766602,9409.942383,9748.158203
3,Brno_1,2024-01-19,9211.941406,8366.894531,8680.223633,8879.445312,9055.506836,9211.941406,9401.630859,9571.526367,9796.778320,10141.545898
4,Brno_1,2024-01-20,10166.933594,9162.351562,9505.362305,9767.815430,9965.879883,10166.933594,10361.991211,10555.380859,10812.354492,11192.518555
...,...,...,...,...,...,...,...,...,...,...,...,...
1183,Prague_3,2024-11-03,5357.871582,5182.174805,5235.636230,5281.838867,5314.794922,5357.871582,5392.748047,5421.582520,5474.044922,5553.786621
1184,Prague_3,2024-11-04,5119.053223,4994.482910,5034.498535,5059.974121,5101.237305,5119.053223,5142.698730,5181.854492,5221.805664,5271.323730
1185,Prague_3,2024-11-05,5203.171875,5045.196289,5118.820801,5147.941895,5175.716309,5203.171875,5251.190918,5275.131348,5317.599121,5387.894043
1186,Prague_3,2024-11-06,5204.630859,5036.324219,5092.420898,5134.061035,5164.396484,5204.630859,5232.375488,5281.375000,5329.729492,5410.321777


### Evaluate Model

In [ ]:
# prompt: Using dataframe forecast_df: keep only first three columns and save as a csv named "timesfmweeklyrohlik.csv"

forecast_df_small = forecast_df.copy() # select first three columns
forecast_df_small.to_csv('timesfmDailyRohlik.csv', index=False) # save as csv, don't save index column


In [ ]:
Y_test_df

,Unnamed: 0,unique_id,ds,holiday,shops_closed,winter_school_holidays,school_holidays,year,month,day,day_of_week,week_of_year,shops_closed_lag,Shop_opened_after_close
0,1133,Brno_1,2024-01-16,0,0,0,0,2024,1,16,1,3,0.0,0
1,1134,Brno_1,2024-01-17,0,0,0,0,2024,1,17,2,3,0.0,0
2,1135,Brno_1,2024-01-18,0,0,0,0,2024,1,18,3,3,0.0,0
3,1136,Brno_1,2024-01-19,0,0,0,0,2024,1,19,4,3,0.0,0
4,1137,Brno_1,2024-01-20,0,0,0,0,2024,1,20,5,3,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,1188,Prague_3,2024-03-11,0,0,0,0,2024,3,11,0,11,0.0,0
236,1189,Prague_3,2024-03-12,0,0,0,0,2024,3,12,1,11,0.0,0
237,1190,Prague_3,2024-03-13,0,0,0,0,2024,3,13,2,11,0.0,0
238,1191,Prague_3,2024-03-14,0,0,0,0,2024,3,14,3,11,0.0,0


In [ ]:
forecast_df['y'] = Y_test_df['y'].values
forecast_df['ds'] = Y_test_df['ds'].values
forecast_df

timesfm   timesfm-0.1   timesfm-0.2   timesfm-0.3  \
unique_id ds                                                                   
Brno_1    2023-05-21  52436.609375  49662.824219  50383.757812  51344.703125   
          2023-05-28  52559.367188  49349.675781  50404.933594  51225.500000   
          2023-06-04  52322.390625  48711.375000  50041.492188  50975.210938   
          2023-06-11  52050.921875  48161.652344  49573.589844  50496.074219   
          2023-06-18  52101.187500  47960.386719  49465.656250  50615.945312   
...                            ...           ...           ...           ...   
Prague_3  2024-02-18  32937.320312  28948.894531  30399.914062  31404.880859   
          2024-02-25  32928.566406  28879.326172  30338.128906  31305.267578   
          2024-03-03  32612.910156  28773.253906  30194.267578  31075.929688   
          2024-03-10  32369.390625  28600.896484  30077.099609  30839.482422   
          2024-03-17  32797.175781  28712.277344  30212.285156  31170.589844   

                       timesfm-0.4   timesfm-0.5   timesfm-0.6   timesfm-0.7  \
unique_id ds                                                                   
Brno_1    2023-05-21  52027.324219  52436.609375  53140.347656  53792.757812   
          2023-05-28  51666.222656  52559.367188  53443.136719  53907.468750   
          2023-06-04  51623.976562  52322.390625  52879.125000  53731.390625   
          2023-06-11  51298.488281  52050.921875  52829.480469  53747.199219   
          2023-06-18  51349.457031  52101.187500  53200.058594  54237.136719   
...                            ...           ...           ...           ...   
Prague_3  2024-02-18  32172.759766  32937.320312  33883.519531  34627.136719   
          2024-02-25  32081.765625  32928.566406  33710.824219  34779.343750   
          2024-03-03  31770.423828  32612.910156  33359.457031  34211.796875   
          2024-03-10  31851.195312  32369.390625  33653.054688  34409.015625   
          2024-03-17  31919.855469  32797.175781  33586.394531  34568.605469   

                       timesfm-0.8   timesfm-0.9      y         ds  
unique_id ds                                                        
Brno_1    2023-05-21  54678.453125  56299.164062  61968 2023-05-21  
          2023-05-28  55023.343750  56684.437500  60661 2023-05-28  
          2023-06-04  54832.246094  56584.828125  58976 2023-06-04  
          2023-06-11  54792.039062  56605.742188  60649 2023-06-11  
          2023-06-18  55458.328125  57699.246094  60003 2023-06-18  
...                            ...           ...    ...        ...  
Prague_3  2024-02-18  35777.421875  37688.746094  62879 2024-02-18  
          2024-02-25  35670.214844  37564.847656  62343 2024-02-25  
          2024-03-03  35481.148438  37075.062500  60905 2024-03-03  
          2024-03-10  35719.351562  37384.417969  61974 2024-03-10  
          2024-03-17  35953.421875  37775.203125  45240 2024-03-17  

[176 rows x 12 columns]

In [ ]:
forecast_df

timesfm   timesfm-0.1   timesfm-0.2   timesfm-0.3  \
unique_id ds                                                                   
Brno_1    2023-05-21  52436.609375  49662.824219  50383.757812  51344.703125   
          2023-05-28  52559.367188  49349.675781  50404.933594  51225.500000   
          2023-06-04  52322.390625  48711.375000  50041.492188  50975.210938   
          2023-06-11  52050.921875  48161.652344  49573.589844  50496.074219   
          2023-06-18  52101.187500  47960.386719  49465.656250  50615.945312   
...                            ...           ...           ...           ...   
Prague_3  2024-02-18  32937.320312  28948.894531  30399.914062  31404.880859   
          2024-02-25  32928.566406  28879.326172  30338.128906  31305.267578   
          2024-03-03  32612.910156  28773.253906  30194.267578  31075.929688   
          2024-03-10  32369.390625  28600.896484  30077.099609  30839.482422   
          2024-03-17  32797.175781  28712.277344  30212.285156  31170.589844   

                       timesfm-0.4   timesfm-0.5   timesfm-0.6   timesfm-0.7  \
unique_id ds                                                                   
Brno_1    2023-05-21  52027.324219  52436.609375  53140.347656  53792.757812   
          2023-05-28  51666.222656  52559.367188  53443.136719  53907.468750   
          2023-06-04  51623.976562  52322.390625  52879.125000  53731.390625   
          2023-06-11  51298.488281  52050.921875  52829.480469  53747.199219   
          2023-06-18  51349.457031  52101.187500  53200.058594  54237.136719   
...                            ...           ...           ...           ...   
Prague_3  2024-02-18  32172.759766  32937.320312  33883.519531  34627.136719   
          2024-02-25  32081.765625  32928.566406  33710.824219  34779.343750   
          2024-03-03  31770.423828  32612.910156  33359.457031  34211.796875   
          2024-03-10  31851.195312  32369.390625  33653.054688  34409.015625   
          2024-03-17  31919.855469  32797.175781  33586.394531  34568.605469   

                       timesfm-0.8   timesfm-0.9      y  
unique_id ds                                             
Brno_1    2023-05-21  54678.453125  56299.164062  61968  
          2023-05-28  55023.343750  56684.437500  60661  
          2023-06-04  54832.246094  56584.828125  58976  
          2023-06-11  54792.039062  56605.742188  60649  
          2023-06-18  55458.328125  57699.246094  60003  
...                            ...           ...    ...  
Prague_3  2024-02-18  35777.421875  37688.746094  62879  
          2024-02-25  35670.214844  37564.847656  62343  
          2024-03-03  35481.148438  37075.062500  60905  
          2024-03-10  35719.351562  37384.417969  61974  
          2024-03-17  35953.421875  37775.203125  45240  

[176 rows x 11 columns]

In [ ]:
mase_seas = partial(mase, seasonality=seasonality)

evaluation_timesfm = evaluate(df=forecast_df, metrics=[mae,mse,smape,mase_seas], train_df =df, models=['timesfm-q-0.5'])
evaluation_timesfm

,unique_id,metric,timesfm-q-0.5
0,Brno_1,mae,1.119854e+04
1,Prague_1,mae,2.007474e+04
2,Prague_2,mae,2.104457e+03
3,Prague_3,mae,2.385128e+04
4,Brno_1,mse,1.628736e+08
5,Prague_1,mse,4.121578e+08
6,Prague_2,mse,8.922285e+06
7,Prague_3,mse,5.977977e+08
8,Brno_1,smape,9.173952e-02
9,Prague_1,smape,2.062707e-01


### Plot predictions

In [ ]:
forecast_df =forecast_df.set_index(['unique_id','ds'])
forecast_df.columns = forecast_df.columns.str.replace(r'^timesfm-q-', 'timesfm-', regex=True)
forecast_df

timesfm   timesfm-0.1   timesfm-0.2   timesfm-0.3  \
unique_id ds                                                                   
Brno_1    2023-05-21  52436.609375  49662.824219  50383.757812  51344.703125   
          2023-05-28  52559.367188  49349.675781  50404.933594  51225.500000   
          2023-06-04  52322.390625  48711.375000  50041.492188  50975.210938   
          2023-06-11  52050.921875  48161.652344  49573.589844  50496.074219   
          2023-06-18  52101.187500  47960.386719  49465.656250  50615.945312   
...                            ...           ...           ...           ...   
Prague_3  2024-02-18  32937.320312  28948.894531  30399.914062  31404.880859   
          2024-02-25  32928.566406  28879.326172  30338.128906  31305.267578   
          2024-03-03  32612.910156  28773.253906  30194.267578  31075.929688   
          2024-03-10  32369.390625  28600.896484  30077.099609  30839.482422   
          2024-03-17  32797.175781  28712.277344  30212.285156  31170.589844   

                       timesfm-0.4   timesfm-0.5   timesfm-0.6   timesfm-0.7  \
unique_id ds                                                                   
Brno_1    2023-05-21  52027.324219  52436.609375  53140.347656  53792.757812   
          2023-05-28  51666.222656  52559.367188  53443.136719  53907.468750   
          2023-06-04  51623.976562  52322.390625  52879.125000  53731.390625   
          2023-06-11  51298.488281  52050.921875  52829.480469  53747.199219   
          2023-06-18  51349.457031  52101.187500  53200.058594  54237.136719   
...                            ...           ...           ...           ...   
Prague_3  2024-02-18  32172.759766  32937.320312  33883.519531  34627.136719   
          2024-02-25  32081.765625  32928.566406  33710.824219  34779.343750   
          2024-03-03  31770.423828  32612.910156  33359.457031  34211.796875   
          2024-03-10  31851.195312  32369.390625  33653.054688  34409.015625   
          2024-03-17  31919.855469  32797.175781  33586.394531  34568.605469   

                       timesfm-0.8   timesfm-0.9      y  
unique_id ds                                             
Brno_1    2023-05-21  54678.453125  56299.164062  61968  
          2023-05-28  55023.343750  56684.437500  60661  
          2023-06-04  54832.246094  56584.828125  58976  
          2023-06-11  54792.039062  56605.742188  60649  
          2023-06-18  55458.328125  57699.246094  60003  
...                            ...           ...    ...  
Prague_3  2024-02-18  35777.421875  37688.746094  62879  
          2024-02-25  35670.214844  37564.847656  62343  
          2024-03-03  35481.148438  37075.062500  60905  
          2024-03-10  35719.351562  37384.417969  61974  
          2024-03-17  35953.421875  37775.203125  45240  

[176 rows x 11 columns]